## Automatically import embryo and predict length of mitosis

In [25]:
import os
import numpy as np
import pandas as pd
import itertools
import time 
import tifffile as tiff
import matplotlib
import matplotlib.pyplot as plt

from IPython.display import display
from numpy import asarray
from time import strftime

import matplotlib.pyplot as plt

from numpy import genfromtxt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, f1_score, recall_score, confusion_matrix, plot_confusion_matrix

import warnings
warnings.filterwarnings('ignore')

from scipy.ndimage import gaussian_filter
from PIL import Image
import cv2
from joblib import dump, load

In [ ]:
def crop_image(file):
    # Remove brightfield and create list of embryos z-stacks    
    file = file[:, 0, :]    
    embryos_list = []
    for i in range(Z_STACKS):
        embryos_list.append(file[i])
        
    # Iterate trough an image and save all 250x250 images to a list
    cropped_images = []
    for embryo in range(len(embryos_list)):
        # Initialize coordinates in the right corner (0, 0 - 256, 256)  
        x1 = 0
        x2 = 256
        y1 = 0
        y2 = 256

        cropped_slices = [] # At the end of iteration it will contain all images
        for i in range (768):
            for j in range (768):
                cropped_slice = embryos_list[embryo][x1: x2, y1: y2]
                cropped_slices.append(cropped_slice)
                x1 += 1
                x2 += 1        
            y1+=1
            y2+=1    
            x1=0
            x2=256   
            
        # Find the brightest images
        sum_values_list = [image.sum() for image in cropped_slices]
        max_image_position = sum_values_list.index(max(sum_values_list))
        cropped_image = cropped_slices[max_image_position]
        cropped_images.append(cropped_image)
    return cropped_images

In [ ]:
def rescale_data(data):
    rescaled_data = []
    for i in range (Z_STACKS):
        a = 255/(data[i].max()-data[i].min())
        c = -a * data[i].min()
        y = np.around(a * data[i] + c)
        rescaled_data.append(y)
    return rescaled_data

In [ ]:
# Changed plt.savefig to cv2.imwrite
def save_as_jpg(image_data):
    os.mkdir(f'embryo_{current_embryo}')
    for i in range(Z_STACKS):
        cv2.imwrite(f'embryo_{current_embryo}/embryo_{current_embryo}_{i}.jpg', image_data[i])

In [ ]:
embryos = [10, 11] # Add embryo position to a list 
Z_STACKS = 85

for i in embryos:
    current_embryo = i
    embryo = tiff.imread(f'/Users/Uzivatel/Documents/Confocal/max_projections/Position {i}_Settings 1_max.tif')

    start_time = time.time()
    data = crop_image(embryo)
    rescaled_data = rescale_data(data)
    save_as_jpg(rescaled_data)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f'It took {execution_time/60} minutes')

In [16]:
directory = f'embryo_{current_embryo}' # File with embryo

images = [] 
for filename in os.listdir(directory): # Add all embryos from directory
    file = os.path.join(directory, filename)
    image = cv2.imread(file)
    images.append(image)
    
flatten_images = [] # Make df from all z-stacks, each row for 1 z-stack
for img in range(len(images)):
    flatten_array = pd.Series(images[img].flatten())
    flatten_images.append(flatten_array)
df_pred = pd.DataFrame(flatten_images)

In [17]:
df_pred

,0,1,2,3,4,5,6,7,8,9,...,196598,196599,196600,196601,196602,196603,196604,196605,196606,196607
0,19,19,19,14,14,14,19,19,19,31,...,12,7,7,7,0,0,0,6,6,6
1,15,15,15,29,29,29,17,17,17,11,...,2,6,6,6,3,3,3,7,7,7
2,4,4,4,8,8,8,4,4,4,10,...,12,13,13,13,3,3,3,7,7,7
3,6,6,6,2,2,2,13,13,13,15,...,13,9,9,9,8,8,8,10,10,10
4,10,10,10,5,5,5,10,10,10,12,...,12,11,11,11,19,19,19,6,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,13,13,13,11,11,11,9,9,9,9,...,1,4,4,4,7,7,7,1,1,1
81,7,7,7,11,11,11,16,16,16,10,...,3,9,9,9,9,9,9,4,4,4
82,16,16,16,7,7,7,7,7,7,10,...,7,8,8,8,9,9,9,4,4,4
83,15,15,15,9,9,9,19,19,19,12,...,8,9,9,9,6,6,6,8,8,8


In [18]:
predictor = load('model_logistic_regression.joblib') 

In [20]:
pred = predictor.predict(df_pred)

In [24]:
pred

array([1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
      dtype=int64)